<a href="https://colab.research.google.com/github/hamisB/machine-learning/blob/master/MarmitonEntreeRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import DATA

In [0]:
# import librairies 
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight") 
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
import ast
%matplotlib inline
import pandas_profiling as pp
import matplotlib.pyplot as plt

In [66]:
# Import sklearn and nltk for NLP processing 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
# Loading the data 
data = pd.read_csv('https://storage.googleapis.com/source_market_fr/data_for_ml/recipe_details.csv')

In [68]:
# Set Title as Index and delelte the columns 'Unnamed: 0
data.set_index('Title', inplace=True)
data.drop('Unnamed: 0', axis=1, inplace=True)
dataset=data.copy()
dataset.head()

,total_time,yiedls,ingredients,instructions,ratings
Title,,,,,
Tacos mexicains,55,4 personnes,"['8 tortillas pour tacos (achetés, ou recette ...","A la poêle, faire dorer l'oignon émincé dans u...",4.6
Velouté de Potiron et Carottes,60,4 personnes,"['1 kg de potiron', '500 g de carotte', '2 pom...","Éplucher et couper en dés le potiron, les pomm...",4.8
Pain (rapide),70,1 pain,"['500 g de farine type 55', '1 sachet de levur...","Mélangez la farine, l'huile d'olive, le sachet...",4.4
Flan de courgettes,50,6 personnes,"['1 kg de courgette', '100 g de gruyère râpé (...",Couper les courgettes en très fines rondelles....,4.8
Gaspacho au Thermomix,20,6 Personnes,"['5 tomates bien mûres (hors saison, prendre d...","Éplucher les tomates (facultatif), un demi-con...",4.1


## Processing ingredients 


In [0]:
# Words to delete 
STOPWORDS = set(nltk.corpus.stopwords.words('french'))
STEMMER = nltk.stem.SnowballStemmer('french')
# Cleaning function
def clean_text(text):
    text_lower = text.lower()
    text_clean = re.sub('[^a-z_]', ' ', text_lower)
    tokens = [w for w in text_clean.split() if (len(w)>2) and (w not in STOPWORDS)]
    tokens_stem = [ STEMMER.stem(token) for token in tokens]
    return tokens_stem

## Vectorizing


In [70]:
tfid = tfidf_vectorizer = TfidfVectorizer(tokenizer=clean_text, ngram_range=(1,2))
tfid_matrix = tfid.fit_transform(dataset['ingredients'])
print(tfid_matrix.shape)
tfid.get_feature_names()[10]

(12780, 48090)


'able'

In [71]:
tfid.get_feature_names()[:10]

['_blank',
 '_blank courget',
 'aaa',
 'aaa cuill',
 'abaiss',
 'abaiss viand',
 'abat',
 'abat cerf',
 'abat eau',
 'abat oignon']

In [0]:
# Calcul similarity 
from sklearn.metrics.pairwise import linear_kernel
similarity = linear_kernel(tfid_matrix, tfid_matrix)

In [0]:
simil_df = pd.DataFrame(similarity, index=dataset.index, columns=dataset.index)

## Recipe Recommender

---



In [91]:
simil_df["Pain farci"].sort_values(ascending=False)[1:11]


Title
'champignons empoisonnés'                              0.331758
Frisée oeuf, jambon et pomme                           0.296075
Oeufs au plat de Virginie aux oignons et aux câpres    0.249449
Salade fraicheur                                       0.247347
Haricots verts vinaigrette aux oeufs durs              0.246318
tomates au thon et chèvre frais                        0.235538
Salade méditerranéenne au thon                         0.230971
Thon tartare aux câpres                                0.222873
Salade poulet avocat et basilic                        0.222020
Salade méchouia algérienne de Coralie                  0.216305
Name: Pain farci, dtype: float64

In [0]:
def recommender(dish):
  print(simil_df[dish].sort_values(ascending=False)[1:11])

In [94]:
recommender("Pissenlits au roquefort en salade")

Title
Mousse aux noix apéritive                        0.528803
Salade d'automne sauce roquefort                 0.434030
Salade de magret de canard aux pommes            0.433583
Salade de courgettes et tomates                  0.389685
Salade de chou rouge aux magrets de Noémie       0.346610
Tarte poire, roquefort et noix                   0.329821
Betteraves aux pommes et aux noix                0.326348
Marinade d'oignon rouge et radis noir            0.324502
Chou rouge au roquefort et aux noix en salade    0.319063
Pissenlits en salade aux lards et aux noix       0.313851
Name: Pissenlits au roquefort en salade, dtype: float64


In [109]:
dropdown = 'Tacos mexicains' #@param ["Tacos mexicains", "Velouté de Potiron et Carottes", "Flan de courgettes"]
print (recommender(dropdown))

Title
Terrine de boeuf aux poivrons                           0.196092
Salade de poivrons (Maroc)                              0.168083
Saumon cru au lait de coco                              0.165544
Terrine rouge et blanc                                  0.158486
Bricks a la viande hachée et aux légumes                0.153009
Keftas                                                  0.146511
Salade de haricots rouges, poulet mariné et crudités    0.143702
Timbale de thon et d'avocat à la tomate                 0.140493
Salade aux haricots rouges, maïs et thon                0.138297
Chilaquiles à a Monique                                 0.136989
Name: Tacos mexicains, dtype: float64
None
